In [1]:
import webhoseio
from datetime import datetime
import json 
import pandas as pd 
from pandas.io.json import json_normalize #package for flattening json in pandas df
webhoseio.config(token="")


In [2]:
start_date_seconds = datetime(2019,10,1,0,0,0).strftime('%s')
start_date_mili = int(start_date_seconds) *1000

In [3]:
start_date_mili

1569884400000

In [ ]:
df = pd.DataFrame()

In [18]:
terms = ['boris johnson', 'jeremy corbyn']
performance_score = 5

In [19]:
# df = pd.DataFrame()

for term in terms:
    query_params = {
    "q": f"language:english site_type:news performance_score:>{performance_score} {term}",
    "ts": start_date_mili,
    "sort": "crawled"}
    print(term)
    result = webhoseio.query("filterWebContent", query_params)
    temp = json_normalize(result['posts'])
    temp['query'] = term
    df = pd.concat([df,temp], axis='index')
    for loop in range(3):
        if result['moreResultsAvailable'] > 100:
            print(term, loop)
            result = webhoseio.get_next()
            temp = json_normalize(result['posts'])
            temp['query'] = term
            df = pd.concat([df,temp], axis='index', sort=False)

boris johnson
boris johnson 0
boris johnson 1
boris johnson 2
jeremy corbyn
jeremy corbyn 0
jeremy corbyn 1
jeremy corbyn 2


In [20]:
df = df.drop_duplicates(subset=['title','author'])
df = df.reset_index(drop=True)
        

In [21]:
df['query'].value_counts()

general election    491
boris johnson       302
jeremy corbyn       147
Name: query, dtype: int64

In [42]:
df.to_pickle('raw_news_data.pkl')

In [22]:
df[['uuid','query','thread.title_full','text','published','thread.site']].to_pickle('election_news_data.pkl')